In [2]:
import pandas as pd
import numpy as np

In [12]:
df1 = pd.read_excel (r'Fall2021_Perceptions_Explanations_All_Final.xlsx')
df2 = pd.read_excel (r'Spring2022_Perceptions_Explanations_All_Final.xlsx')
dfCornell = pd.read_csv("Coded_Cornell1110_LabPerceptions_Edgelist.csv")
dfNCState = pd.read_csv("Coded_NCStatePY206_LabPerceptions_Edgelist.csv")
dfUTAustin = pd.read_csv("Coded_UTAustinPHY 105M_LabPerceptions_Edgelist.csv")

df = pd.concat([df1,df2])
df = df[df["misplaced"].isnull()]
df = df[df["explanation"].notnull()]

dfCornell = dfCornell[dfCornell["explanation"].notnull()]
dfNCState = dfNCState[dfNCState["explanation"].notnull()]
dfUTAustin = dfUTAustin[dfUTAustin["explanation"].notnull()]


In [22]:
def define_code_col(df_dum,code):
    if "code5" in df_dum:
        df_dum[code] = np.where(df_dum["code1"] == code,1,0) + np.where(df_dum["code2"] == code,1,0) + np.where(df_dum["code3"] == code,1,0)+ np.where(df_dum["code4"] == code,1,0)+ np.where(df_dum["code5"] == code,1,0)
    else: 
        df_dum[code] = np.where(df_dum["code1"] == code,1,0) + np.where(df_dum["code2"] == code,1,0) + np.where(df_dum["code3"] == code,1,0)+ np.where(df_dum["code4"] == code,1,0)
    return df_dum


In [25]:
codes = ["CON","EXT","PER","NAT","MOT","ANA","PROB","PLAN","EXEC","WRIT","PART","LEAD","EXP","HELP","OTH"]

for code in codes:
    df = define_code_col(df,code)
    dfCornell = define_code_col(dfCornell,code)  
    dfUTAustin = define_code_col(dfUTAustin,code)
    dfNCState = define_code_col(dfNCState,code)

In [30]:
dfCornell["from"] = ""
dfCornell["to"] = ""
dfUTAustin["from"] = ""
dfUTAustin["to"] = ""
dfNCState["from"] = ""
dfNCState["to"] = ""

In [33]:
df["Group"] = "Train:Cornell"
dfCornell["Group"] = "Cornell"
dfUTAustin["Group"] = "UT"
dfNCState["Group"] = "NCState"

In [34]:
df = pd.concat([df, dfCornell, dfUTAustin, dfNCState])

In [35]:
df

,context,explanation,misplaced,code1,code2,code3,code4,code5,CON,EXT,...,EXEC,WRIT,PART,LEAD,EXP,HELP,OTH,Group,from,to
0,lab,He has a great grasp of all of the different q...,NaN,CON,ANA,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,Train:Cornell,NaN,NaN
1,lab,Their concepts are very strong and they always...,NaN,CON,PLAN,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,Train:Cornell,NaN,NaN
2,lab,Focused on his work.,NaN,MOT,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,Train:Cornell,NaN,NaN
3,lab,Has great understanding of software to help an...,NaN,ANA,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,Train:Cornell,NaN,NaN
4,lab,Works at air and space museum,NaN,EXT,NaN,NaN,NaN,NaN,0,1,...,0,0,0,0,0,0,0,Train:Cornell,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,NaN,is able to complete labs efficiently and effec...,NaN,MOT,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,NCState,,
291,NaN,He completes all his work adequately and respe...,NaN,MOT,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,0,0,0,NCState,,
292,NaN,He portrays a student of excellence and determ...,NaN,CON,MOT,NaN,NaN,NaN,1,0,...,0,0,0,0,0,0,0,NCState,,
293,NaN,My lab partner. Absolutely carried the math lo...,NaN,ANA,HELP,NaN,NaN,NaN,0,0,...,0,0,0,0,0,1,0,NCState,,
